![__Predict the age of abalone from physical measurements.__
Sex - nominal - M, F, and I (infant),
Length - Bcontinuous - mm - Longest shell measurement,
Diameter - continuous - mm - perpendicular to length,
Height - continuous - mm - with meat in shell,
Whole weight - continuous - grams - whole abalone,
Shucked weight - continuous - grams - weight of meat,
Viscera weight - continuous - grams - gut weight (after bleeding),
Shell weight - continuous - grams - after being dried,
Rings - integer - +1.5 gives the age in years (target).](https://cdn.shopify.com/s/files/1/0016/6959/5189/files/juvenile-red-abalone-shells.jpg?v=1600820403)


#__ИМПОРТ И ЗАГРУЗКА__

__Импортируем необходимые библиотеки__

In [2]:
import pandas as pd,numpy as np

__Загружаем очищенный датасет, полученный в результате работы EDA и фиксируем параметр воспроизводимости__ 

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/bakenbardich/AI-Education/main/EDA/data/abalone_clean.csv')
RANDOM_STATE=101

In [4]:
df.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.7985,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4177 non-null   object 
 1   Length          4177 non-null   float64
 2   Diameter        4177 non-null   float64
 3   Height          4177 non-null   float64
 4   Whole weight    4177 non-null   float64
 5   Shucked weight  4177 non-null   float64
 6   Viscera weight  4177 non-null   float64
 7   Shell weight    4177 non-null   float64
 8   Rings           4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


#__ПОСТРОЕНИЕ МОДЕЛЕЙ НА ЧИСЛОВЫХ ПРИЗНАКАХ__

__Naive Bayes__

In [6]:
#Импортируем необходимые библиотеки. 
from sklearn.model_selection import train_test_split
#У нас задача предсказания возраста ушка-задача регрессии, используем Байеса для регрессии.
from sklearn.linear_model import BayesianRidge
#Сразу импортируем несколько метрик, учитывая поставленную задачу для оценки качества работы модели.
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error,mean_squared_error

In [7]:
#Делим датасет на матрицу объект-признак(тут также исключаем категориальный признак) и таргет.
X = df.drop(['Rings','Sex'],axis=1)
y = df['Rings']

In [21]:
#Перед построением моделей посмотрим на распределение таргета.
y.hist(backend='plotly',xlabelsize=10,ylabelsize=8,title='РАСПРЕДЕЛЕНИЕ ЦЕЛЕВОЙ ПЕРЕМЕННОЙ')

__Распределение таргета приближено к нормальному распределению,что позитивно отразится на обучении.__

In [22]:
#Делим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

In [23]:
#Создадим класс модели регрессии Байеса(параметры пока по умолчанию) и обучим на тренировочной выборке.
NB_model = BayesianRidge()
NB_model.fit(X_train,y_train)

BayesianRidge()

In [24]:
#Предсказание на тестовой выборке.
NB_pred = NB_model.predict(X_test)

In [28]:
#Оценка качества модели
MAE = mean_absolute_error(y_test,NB_pred)
MAPE = mean_absolute_percentage_error(y_test,NB_pred)
MSE = mean_squared_error(y_test,NB_pred)
print("MAE:", round(MAE,4))
print(f"MAPE: {round(MAPE,4) * 100:4f} %")
print("MSE:", round(MSE,4))

MAE: 1.6205
MAPE: 16.600000 %
MSE: 5.6303


__Считаю, что разница между фактическим значением и предсказанным даст нам более объективное представление, чем другие метрики, так как, например ,MAE = 1.6205 - не понятно как интерпретировать, много это или мало для возраста ушек, поэтому будем использовать MAPE, так как данная метрика показывает процентное отношение фактического возраста к предсказанному__  

__K-Nearest Neighbors, KNN__

In [29]:
#Импортируем необходимые библиотеки для KNN.
from sklearn.neighbors import KNeighborsRegressor
#Импортируем библиотеку для масштабирования данных, которая необходима для KNN, корректного  подсчета расстояния между объектами.
from sklearn.preprocessing import StandardScaler

In [30]:
#Масштабируем признаки. X_test -делаем только transform, чтобы при fit не допустить утечки данных.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [31]:
#Создаём модель KNN, обучаем на масштабированных данных, делаем предикт.
KNN_model = KNeighborsRegressor()
KNN_model.fit(X_train_scaled,y_train)
KNN_pred = KNN_model.predict(X_test_scaled)

In [32]:
#Оцениваем качество модели, используя так же метрику MAPE.
mean_absolute_percentage_error(y_test,KNN_pred)

0.15948210909354812

__KNN + GridSearch__

In [33]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_neighbors': range(1, 10),
    'weights': ['uniform', 'distance'],  # 'uniform' - все соседи равнозначны, 'distance' - веса обратно пропорциональны расстоянию
    'metric': ['euclidean', 'manhattan']  # Различные метрики расстояния
}

# Создаем объект GridSearchCV
grid_search = GridSearchCV(KNN_model, param_grid, cv=5, scoring='neg_mean_absolute_percentage_error', verbose=1)

# Проводим поиск по сетке с кросс-валидацией
grid_search.fit(X_train_scaled, y_train)

# Выводим лучшие параметры и результаты
print("Best params:", grid_search.best_params_)
print("Best MAPE on train data:", grid_search.best_score_)

# Делаем предсказания на тестовых данных с лучшей моделью
best_knn_model = grid_search.best_estimator_
y_pred = best_knn_model.predict(X_test_scaled)

# Оцениваем модель на тестовых данных
mean_absolute_percentage_error(y_test, y_pred)


Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best params: {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}
Best MAPE on train data: -0.15575854080046517


0.15104691664523937

__Получилось немного снизить ошибку__

__Random Forest Regressor__

In [34]:
#Импортируем библиотеку для работы со случайным лесом.
from sklearn.ensemble import RandomForestRegressor

In [37]:
#Создаем модель леса, обучаем на тренировочных данных, делаем предсказание на тесте, оцениваем качество 
RF_model = RandomForestRegressor(random_state=RANDOM_STATE)
RF_model.fit(X_train,y_train)
RF_pred = RF_model.predict(X_test)
mean_absolute_percentage_error(y_test,RF_pred)

0.15571073520718592

In [39]:
#Подбираем гиперпараметры для леса с помощью поиска по сетке.
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth': [5,10,50],
              'n_estimators':[100,150,200],
              'max_features':['sqrt','log2']}
gs = GridSearchCV(estimator=RF_model,
                  param_grid=param_grid,
                  cv=5,
                  scoring='neg_mean_absolute_percentage_error',
                  verbose=2,
                  n_jobs=-1)
gs.fit(X_train,y_train)
RF_GS_pred = gs.best_estimator_.predict(X_test)
mean_absolute_percentage_error(y_test, RF_GS_pred)              

Fitting 5 folds for each of 18 candidates, totalling 90 fits


0.15019734008479269

__Итак: KNN и RandomeForest дали примерно одинаковые результаты, но второй незначительно лучше, давайте работать далее именно с ним.__

#__ДОБАВЛЕНИЕ КАТЕГОРИЛЬНОГО ПРИЗНАКА В ЛУЧШУЮ МОДЕЛЬ И ПОДБОР ГИПЕРПАРАМЕТРОВ__

__Прежде чем брать за лучшую модель Random Forest, хочу попробывать оценить еще одну модель, впечатлившую меня с Практикума по ML -модификацию градиентного бустинга CatBoost. Одно из ее преимуществ это то ,что указав список индексов категориальных фичей, эта модель сама будет кодировать их.
После оценки качества с параметрами по умолчанию, буду осуществлять подбор гиперпараметров через Optuna.__

In [40]:
#Импортируем библиотеку для работы с CatBoost
from catboost import CatBoostRegressor

In [41]:
#Переинициализируем матрицу объект-признак, добавив назад категориальную фичу "SEX"
X = df.drop('Rings',axis=1)
y = df['Rings']

In [42]:
#Заново разделим на трейн и тест 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

In [43]:
CBR_model = CatBoostRegressor(cat_features=[0],custom_metric='MAPE')
CBR_model.fit(X_train,y_train)
CBR_pred = CBR_model.predict(X_test)
mean_absolute_percentage_error(y_test,CBR_pred)

Learning rate set to 0.049036
0:	learn: 3.1737974	total: 179ms	remaining: 2m 59s
1:	learn: 3.1175201	total: 195ms	remaining: 1m 37s
2:	learn: 3.0643772	total: 209ms	remaining: 1m 9s
3:	learn: 3.0168652	total: 222ms	remaining: 55.3s
4:	learn: 2.9715676	total: 235ms	remaining: 46.8s
5:	learn: 2.9283863	total: 250ms	remaining: 41.5s
6:	learn: 2.8866119	total: 266ms	remaining: 37.8s
7:	learn: 2.8500884	total: 280ms	remaining: 34.7s
8:	learn: 2.8146703	total: 295ms	remaining: 32.5s
9:	learn: 2.7792702	total: 309ms	remaining: 30.6s
10:	learn: 2.7481738	total: 322ms	remaining: 28.9s
11:	learn: 2.7184315	total: 334ms	remaining: 27.5s
12:	learn: 2.6932782	total: 348ms	remaining: 26.4s
13:	learn: 2.6711796	total: 359ms	remaining: 25.3s
14:	learn: 2.6466139	total: 371ms	remaining: 24.3s
15:	learn: 2.6254308	total: 382ms	remaining: 23.5s
16:	learn: 2.6019245	total: 395ms	remaining: 22.8s
17:	learn: 2.5802097	total: 407ms	remaining: 22.2s
18:	learn: 2.5589027	total: 421ms	remaining: 21.7s
19:	learn

0.1528157258850025

In [44]:
# У данной модификации бустинга очень большой ассортимент полезных опций, воспользуемся одной из них и заодно проанализируем важность признаков.
feature_importances = CBR_model.get_feature_importance()
feature_names = X_train.columns
for score,name in   sorted(zip(feature_importances,feature_names),reverse=True):
    print('{}: {}'.format(name,score))

Shell weight: 28.668380020221775
Shucked weight: 22.62607673315495
Height: 10.476296761671339
Whole weight: 10.093386843181055
Viscera weight: 7.498819126936883
Diameter: 7.235183532379406
Length: 7.111445337548261
Sex: 6.290411644906319


__Как видими топ 2 по важности признака Shell weight и Shucked weight , а вот добавленный SEX оказывает меньше всего значения на предсказание.__ 

__Отлично Catboost со значениями по умолчанию уменьшил ошибку на пол процента, пробуем подобрать гиперпараметры через Optuna__ 

In [46]:
import optuna

In [51]:
def objective(trial):
    param = {"n_estimators": trial.suggest_int("n_estimators", 50, 2500),
              "max_depth": trial.suggest_int("max_depth", 2, 16),
              "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
              "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 1, 30),
              "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
              "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
              "learning_rate": trial.suggest_float("learning_rate", 0.01, 1)}
    estimator = CatBoostRegressor(**param,verbose=False,cat_features=[0])
    estimator.fit(X_train, y_train)
    pred = estimator.predict(X_test)
    return mean_absolute_percentage_error(y_test,pred)
study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials=100)
print(study.best_trial)
CBR_model_optuna = CatBoostRegressor(**study.best_params,cat_features=[0])
CBR_model_optuna.fit(X_train,y_train)
CBR_pred= CBR_model_optuna.predict(X_test)
mean_absolute_percentage_error(y_test, CBR_pred)

[I 2024-04-03 21:19:49,357] A new study created in memory with name: no-name-65e78ef9-e436-4233-8b61-ddd6c9cf6fe6
[I 2024-04-03 21:19:50,550] Trial 0 finished with value: 0.1612486874973293 and parameters: {'n_estimators': 1183, 'max_depth': 6, 'boosting_type': 'Plain', 'l2_leaf_reg': 12, 'bootstrap_type': 'MVS', 'colsample_bylevel': 0.015431671278867003, 'learning_rate': 0.7053593481695462}. Best is trial 0 with value: 0.1612486874973293.
[I 2024-04-03 21:19:52,488] Trial 1 finished with value: 0.15532655298249123 and parameters: {'n_estimators': 2229, 'max_depth': 16, 'boosting_type': 'Plain', 'l2_leaf_reg': 24, 'bootstrap_type': 'Bayesian', 'colsample_bylevel': 0.04006950806832424, 'learning_rate': 0.4311924455161436}. Best is trial 1 with value: 0.15532655298249123.
[I 2024-04-03 21:19:56,431] Trial 2 finished with value: 0.15694450457914078 and parameters: {'n_estimators': 2372, 'max_depth': 4, 'boosting_type': 'Ordered', 'l2_leaf_reg': 12, 'bootstrap_type': 'Bayesian', 'colsample

FrozenTrial(number=71, state=1, values=[0.1500007441493955], datetime_start=datetime.datetime(2024, 4, 3, 21, 23, 20, 509807), datetime_complete=datetime.datetime(2024, 4, 3, 21, 23, 24, 580512), params={'n_estimators': 1673, 'max_depth': 15, 'boosting_type': 'Plain', 'l2_leaf_reg': 26, 'bootstrap_type': 'Bayesian', 'colsample_bylevel': 0.0894141891841484, 'learning_rate': 0.18564566846366976}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=2500, log=False, low=50, step=1), 'max_depth': IntDistribution(high=16, log=False, low=2, step=1), 'boosting_type': CategoricalDistribution(choices=('Ordered', 'Plain')), 'l2_leaf_reg': IntDistribution(high=30, log=False, low=1, step=1), 'bootstrap_type': CategoricalDistribution(choices=('Bayesian', 'Bernoulli', 'MVS')), 'colsample_bylevel': FloatDistribution(high=0.1, log=True, low=0.01, step=None), 'learning_rate': FloatDistribution(high=1.0, log=False, low=0.01, step=None)}, trial_id=71

0.1500007441493955

__Отлично, с помощью подбора гиперпараметров так же улучшили метрику .__

__Вычисление дополнительных метрик и их интерпретация__

In [52]:
#Импортируем необходимые метрики.
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# Делаем предсказания на тренировочных данных с лучшей моделью
y_train_pred = CBR_model_optuna.predict(X_train)
# Вычисляем метрики для тренировочных данных
mae_train = mean_absolute_error(y_train, y_train_pred)
mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
r2_train = r2_score(y_train, y_train_pred)
# Вывод результатов для тренировочных данных
print('Train data:')
print(f'MAE: {mae_train:.4f}')
print(f'MSE: {mse_train:.4f}')
print(f'RMSE: {rmse_train:.4f}')
print(f'R2: {r2_train:.4f}')

print()

# Вывод результатов для тестовых данных
print('Test data:')
mae = mean_absolute_error(y_test, CBR_pred)
print(f'MAE: {mae:.4f}')
mse = mean_squared_error(y_test, CBR_pred)
print(f'MSE: {mse:.4f}')
rmse = np.sqrt(mean_squared_error(y_test, CBR_pred))
print(f'RMSE: {rmse:.4f}')
r2 = r2_score(y_test, CBR_pred)
print(f'R2: {r2:.4f}')

Train data:
MAE: 1.3481
MSE: 3.4430
RMSE: 1.8555
R2: 0.6707

Test data:
MAE: 1.4784
MSE: 4.5761
RMSE: 2.1392
R2: 0.5511


__Выводы по метрикам:__

__MAE: 1.3481 говорит о том, что модель в среднем ошибается на примерно чуть больше чем год при предсказании возраста ушек на тренировочных данных и на MAE: 1.4784 на тестовых. В целом результат на тестовых и тренировочных данных сравним, так что можно сказать что модель имеют неплохую обобщающую способность.__

__RMSE: 1.8555 - корень из средней квадратичной ошибки RMSE Train и RMSE 2.1392 Test. Значение Test выше, чем на тренировочных данных, что может указывать на некоторое переобучение, но оно, кажется, все еще довольно низкое.__

__Train data R2: 0.6707 - говорит о том, что модель "угадывает" примерно 67.07% изменчивости в данных для тренировочного набора. Это средний показатель показатель. Test data R2: R2: 0.5511 - на тестовых данных модель "угадывает" примерно 55.11% изменчивости. Это тоже приемлемый результат, но он ниже, чем на тренировочных данных, что может сигнализировать о некотором переобучении или о том, что модель несколько хуже справляется с новыми данными,но разница не слишком большая,чтобы говорить о сильном оферфите.__

__Модель показывает адекватные результаты как на тренировочных данных , так и на тестовых, разница между метриками минимальна.  На тестовых данных модель  обладает приемлемой обобщающей способностью, и ее прогнозы могут быть полезны в практическом применении.__

#__Explainer Dashboard__

In [54]:
from explainerdashboard import RegressionExplainer,ExplainerDashboard

In [55]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [57]:
# Создаем Explainer
explainer = RegressionExplainer(CBR_model_optuna, X_test, y_test)

# Создаем ExplainerDashboard
db = ExplainerDashboard(explainer, title="CatBoost Explainer Dashboard",
                        whatif=False).run()

Generating self.shap_explainer = shap.TreeExplainer(model)
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
For this type of model and model_output interactions don't work, so setting shap_interaction=False...
The explainer object has no decision_trees property. so setting decision_trees=False...
Generating layout...
Calculating shap values...
Calculating predictions...
Calculating residuals...
Calculating absolute residuals...
Calculating dependencies...
Calculating importances...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard on http://192.168.1.4:8050


C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.


ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=8050): Max retries exceeded with url: /_alive_f9231a1a-7243-46c2-a857-d63c517452f8 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000029FA8433980>: Failed to establish a new connection: [WinError 10049] Требуемый адрес для своего контекста неверен'))

#__Анализ модели в Explainer Dashboard__

__По данным explainer dashboard можно сказать, что наибольший вклад в результат по SHAP вносят Shell weight и Shucked weight.__

__Метрики
Model Summary
Quantitative metrics for model performance
metric	Score
mean-squared-error	4.576
root-mean-squared-error	2.139
mean-absolute-error	1.478
mean-absolute-percentage-error	0.15
R-squared	0.551__

__Mean Square Error (MSE):
MSE вычисляет среднеквадратичную ошибку между фактическими и предсказанными значениями.
Чем меньше значение MSE, тем лучше модель предсказывает данные.
Root Mean Squared Error (RMSE):
RMSE является корнем из MSE и представляет собой среднеквадратичное отклонение предсказанных значений от фактических.
Чем меньше значение RMSE, тем лучше модель предсказывает данные.
Mean Absolute Error (MAE):
MAE вычисляет среднее абсолютное отклонение между фактическими и предсказанными значениями.
Чем меньше значение MAE, тем лучше модель предсказывает данные.
Mean Absolute Percentage Error (MAPE):
MAPE вычисляет среднее абсолютное отклонение в процентном соотношении между фактическими и предсказанными значениями.
Чем меньше значение MAPE, тем лучше модель предсказывает данные.
Значение MAPE равное 0,15 означает, что среднее абсолютное отклонение в процентном соотношении между предсказанными и фактическими значениями составляет примерно 15%.
R-squared (R^2):
R^2 (коэффициент детерминации) показывает долю дисперсии целевой переменной, объясненную моделью.
Значение R^2 ближе к 1 означает лучшее соответствие модели данным.
Значение R^2 равное 0,551 означает, что модель объясняет примерно 55,1% дисперсии целевой переменной.__



__Анализ 2-3 индивидуальных прогнозов с комментарием__

__индекс 1791
Predicted 10,732
Observed 10,000
На графике Contribution Plot можно визульно пронаблюдать какой именно вклад вносит каждая из фичей в числовом значении. Интересно, что вляние фич отсчитывается от Average of Population, что видимо какое-то вычисленное начальное значение. Для индекса 1791 интересно, что shucked weight понижает возраст, а shell weight повышает.__

__индекс 1764
Predicted	5.274
Observed	5.000
В данном случае почти все фичи двигают значение возраста в сторону уменьшения, кроме shucked weight__